In [1]:
import tensorflow as tf
from os import path, getcwd, chdir
import zipfile
path = f"{getcwd()}/utf-8''happy-or-sad.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

In [15]:
def train_happy_sad_model():
    
    DESIRED_ACCURACY = 0.999
    
    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get("accuracy")>DESIRED_ACCURACY):
                print("\nReached 99.9% accuracy so cancelling training!")
                self.model_stop_training = True
    callbacks=myCallback()
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation = "relu"),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation = "relu"),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation=tf.nn.relu),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    
    from tensorflow.keras.optimizers import RMSprop
    
    model.compile(loss = "binary_crossentropy",
             optimizer=RMSprop(lr=0.001),
             metrics=["accuracy"])
    
    model.summary()
    
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    
    train_datagen = ImageDataGenerator(rescale = 1./255)

    train_dir = '/tmp/h-or-s'

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=8,
        class_mode='binary')
    
    history = model.fit(
        train_generator, 
        steps_per_epoch=10, 
        epochs=20,
        verbose=2,
        callbacks=[callbacks])
    return history.history['accuracy'][-1]

In [16]:
train_happy_sad_model()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 18496)            

1.0